<i><b>Public AI</b></i>
<br>

# 텐서플로우로 구현하는 베이지안 개인화 랭킹

이전 주차에 우리는 텐서플로우로 심층신경망을 구현하는 방법을 배웠습니다. 그리고 3주차에서는 Matrix Factorization하는 방법으로 Bayesian Personalized Ranking을 이용하여 고객에 대한 Embedding 정보, 아이템에 대한 Embedding 정보를 획득하는 방법을 배웠었지요. 이번 시간에는 이전에 배웠던 BPR 알고리즘을 텐서플로우로 구현하는 프로젝트를 통해 텐서플로우의 동작을 더욱 잘 이해해 봅시다.

### _Objective_
1. **BPR을 텐서플로우로 구현하기** : Tensorflow로 Bayesian Personalized Ranking을 구성하는 방법을 배워봅니다.


In [1]:
%matplotlib inline
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.utils import get_file
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Embedding
np.set_printoptions(5,)

# \[ 데이터 셋 : MovieLens 100K\]
---


## 1. 데이터 셋 가져오기

In [2]:
ROOT_URL = "https://pai-datasets.s3.ap-northeast-2.amazonaws.com/recommender_systems/movielens_100k/datasets/"

# 데이터 가져오기
ratings_path = get_file("100k_ratings.csv", ROOT_URL+"ratings.csv")
movies_path = get_file("100k_movies.csv",ROOT_URL+"movies.csv")
users_path = get_file("100k_users.csv", ROOT_URL+"users.csv")

ratings_df = pd.read_csv(ratings_path)
movies_df = pd.read_csv(movies_path)
users_df = pd.read_csv(users_path)

## 2. 데이터 확인하기

### (1) ratings_df 데이터셋
+ user_id : user 식별자
+ item_id : 영화(Item) 식별자
+ rating : 각 user별 영화의 평가 점수

In [3]:
print("ratings_df의 크기 : ", ratings_df.shape)

# 다섯개 데이터를 Random으로 가져옴
ratings_df.sample(5, random_state=1)

ratings_df의 크기 :  (99991, 3)


,user_id,item_id,rating
27819,358,529,3
86043,806,879,3
14316,286,25,3
91233,378,660,4
28761,311,12,4


### (2) movies_df 데이터셋

+ item_id : 영화(item) 식별자
+ title : 영화(item) 제목
+ year : 영화 개봉 년도
+ unknown ~ Western : 영화 장르의 멀티핫인코딩(Multi-hot encoding)

In [4]:
print("movies_df의 크기 : ", movies_df.shape)

# 5개를 Random으로 가져옴
movies_df.sample(5, random_state=1)

movies_df의 크기 :  (1681, 22)


,item_id,title,year,unknown,Action,Adventure,Animation,Children,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1335,1337,Larger Than Life (1996),1996,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1340,1342,"Convent, The (Convento, O) (1995)",1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
288,290,Fierce Creatures (1997),1997,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
301,303,Ulee's Gold (1997),1997,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1674,1676,"War at Home, The (1996)",1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### (3) users_df 데이터셋

+ user_id : user 식별자
+ age : user의 나이 (0\~4세 :0, 5\~9세:1, ... )
+ gender : user의 성별
+ occupation : user의 직업군

In [5]:
print("users_df의 크기 : ", users_df.shape)

# 5개를 Random으로 가져옴
users_df.sample(5, random_state=1)

users_df의 크기 :  (943, 4)


,user_id,age,gender,occupation
785,786,7,F,engineer
94,95,6,M,administrator
265,266,12,F,administrator
688,689,5,M,other
859,860,14,F,retired


## 3. K-Core Sampling을 통해 user_id, item_id 추려내기

k=5를 기준으로, 너무나 적게 평가하거나 평가받은 유저 및 영화는 제거하도록 하겠습니다.

In [6]:
like_df = ratings_df.copy()

threshold = 5

count = 0
while True:
    prev_total = len(like_df)
    print(f"{count}회차 데이터 수 : {prev_total:,}개")
    
    total_user_per_item = (
        like_df
        .groupby('item_id')['user_id']
        .count())
    over_item_ids = total_user_per_item[
        total_user_per_item>threshold].index
    
    total_item_per_user = (
        like_df
        .groupby('user_id')['item_id']
        .count())
    over_user_ids = total_item_per_user[
        total_item_per_user>threshold].index
    
    like_df = like_df[
        (like_df.user_id.isin(over_user_ids))
        &(like_df.item_id.isin(over_item_ids))]

    if prev_total == len(like_df):
        print("종료")
        break
    count += 1

0회차 데이터 수 : 99,991개
1회차 데이터 수 : 99,023개
종료


## 4. 평가 기준 설정하기

Bayesian Personalized Ranking과 Neural Collaborative Filtering의 성능을 비교해보기 위해 우리는 Hit Ratio를 이용하도록 하겠습니다.

### (1) 평가데이터셋 구성하기

우선 Train 데이터셋와 Test 데이터셋을 나누도록 하겠습니다. Test 데이터셋은 각 고객이 평가한 마지막 영화로 두도록 하겠습니다.

In [7]:
trains = []
tests = []
for i, group_df in like_df.groupby('user_id'):
    # 마지막 직전은 Train_, 미자믹은 test_
    train_, test_ = group_df.iloc[:-1], group_df.iloc[-1:]
    trains.append(train_)
    tests.append(test_)
    
train_df = pd.concat(trains)
test_df = pd.concat(tests)

# user_id를 기준으로 정렬된 것을 무작위로 섞음
train_df = train_df.sample(frac=1)
test_df = test_df.sample(frac=1)

In [8]:
print("train 데이터 셋의 수 : ", len(train_df))
print("test 데이터 셋의 수 : ", len(test_df))

train 데이터 셋의 수 :  98080
test 데이터 셋의 수 :  943


### (2) Hit Ratio를 위한 데이터셋 구성하기 

이번 시간에는 평가지표를 Hit Ratio를 이용해보도록 하겠습니다.<br>

Hit Ratio의 측정방법은 아래와 같습니다.

> 고객이 구매한 아이템 1개와 고객이 구매하지 않은 아이템 100개를 가져온 후, 고객이 구매한 아이템 고객이 구매한 아이템이 101개 중 몇번째에 위치하는지를 확인하기. Top-10 Hit Ratio란, 고객이 구매한 아이템이 10번째 안에 들어있는 확률로, 높을수록 보다 정확하게 추천한다고 판단

In [9]:
# 유저별 평가한 영화목록 구성하기
itemset_per_user = (
    ratings_df
    .groupby('user_id')
    ['item_id']
    .apply(frozenset)
)

total_items = set(ratings_df.item_id.unique())

# 유저가 평가하지 않은 영화목록 구성하기
notseen_itemset_per_user = total_items - itemset_per_user
notseen_itemset_per_user = notseen_itemset_per_user.apply(list)

hit ratio를 계산하기 위해서 우리는 본 영화 1개와 보지 않은 영화 100개를 구성해야 합니다. 그리고 본 영화와 보지 않은 영화 모두 모델로 추론한 후, 선호도 순서대로 정렬 후 본 영화가 10등 안에 들었으면, 모델이 올바르게 추론했다고 평가하고, 들지 않으면 모델이 잘못 추론했다고 평가하는 방식입니다.

In [10]:
hit_ratio_df = test_df.copy()

hit_ratio_df['not_seen_list'] = hit_ratio_df.user_id.apply(
    lambda x : random.choices(notseen_itemset_per_user[x],k=100))

hit_ratio_df = hit_ratio_df.drop('rating',axis=1)
hit_ratio_df.head()

,user_id,item_id,not_seen_list
87148,390,690,"[176, 1415, 1148, 393, 208, 1525, 1204, 1104, ..."
99729,32,408,"[1568, 498, 1124, 1378, 1365, 338, 800, 420, 1..."
97438,534,508,"[833, 963, 513, 1607, 423, 72, 1612, 317, 440,..."
99644,64,184,"[617, 955, 629, 75, 1385, 946, 904, 274, 983, ..."
95908,47,262,"[610, 201, 141, 377, 457, 1351, 895, 381, 861,..."


In [11]:
unseen_items = []
for user_id, group_df in tqdm(ratings_df.groupby('user_id')):
    
    sampled_items = (ratings_df
                     .loc[~ratings_df.item_id.isin(set(group_df.item_id)), 'item_id']
                     .sample(200)
                     .drop_duplicates()
                     .iloc[:100]
                     .tolist()
                    )
    assert len(sampled_items) == 100
    unseen_items.append((user_id, sampled_items))

100%|██████████| 943/943 [00:03<00:00, 300.44it/s]


In [12]:
hit_ratio_df = test_df.copy()
unseen_df = pd.DataFrame(unseen_items, columns=['user_id', 'not_seen_list'])
hit_ratio_df = hit_ratio_df.merge(unseen_df, on='user_id')

In [13]:
del hit_ratio_df['rating']
hit_ratio_df.head()

,user_id,item_id,not_seen_list
0,390,690,"[325, 134, 188, 780, 7, 465, 193, 428, 880, 80..."
1,32,408,"[426, 748, 203, 596, 1028, 392, 615, 1054, 693..."
2,534,508,"[430, 8, 401, 229, 56, 272, 523, 26, 152, 781,..."
3,64,184,"[411, 178, 1110, 724, 637, 443, 1017, 483, 250..."
4,47,262,"[483, 237, 502, 239, 427, 520, 514, 191, 245, ..."


# \[ 1. Bayesian Personalized Ranking 구성하기 \]
---

이전 시간에 배운 Bayesian Personalized Ranking을 텐서플로우로 작성해보도록 하겠습니다.

## 1. 모델 구성하기

### (1) Input 구성하기

Bayesian Personalized Ranking의 핵심 아이디어는 바로 

> 고객이 본 영화는 고객이 보지 않은 영화보다 항상 선호도가 높다

입니다. Bayesain Personalized Ranking에서는 고객과 고객이 본 영화, 그리고 고객이 보지 않은 영화 총 3 개의 입력이 들어가게 됩니다.

In [14]:
from tensorflow.keras.layers import Input

user_id = Input(shape=(), name='user')  # name : user
pos_item_id = Input(shape=(), name='positive_item') # name : positive_item 
neg_item_id = Input(shape=(), name='negative_item') # name : negative_item

### (2) 임베딩 레이어 구성하기

우리가 Bayesian Personalized Ranking에서 해야하는 것은 상호작용 정보를 통해 유저와 아이템에 대한 적절한 임베딩 값을 추론하는 것에 있습니다. 아래와 같이 임베딩 레이어를 생성하도록 하겠습니다.
<br>
이 때 Item Embedding의 경우, 각 아이템 별 편향 정보(Bias)를 추가하기 위해 Num Factor에 1을 더하도록 하겠습니다.

In [15]:
from tensorflow.keras.layers import Embedding

num_user = ratings_df.user_id.max() + 1
num_item = ratings_df.item_id.max() + 1
num_factor = 30

user_embedding_layer = Embedding(num_user, num_factor, name='user_embedding')
item_embedding_layer = Embedding(num_item, num_factor + 1, name='item_embedding')

In [16]:
user_embedding_layer.build(input_shape=())
W = user_embedding_layer.get_weights()
W[0][1]

array([-0.00124,  0.02806,  0.01952,  0.01807, -0.01903,  0.02258,
        0.03024, -0.04098, -0.02558, -0.00899, -0.01783, -0.04243,
       -0.01648,  0.02209, -0.00038,  0.00113, -0.04222,  0.00695,
        0.04144,  0.04491,  0.0142 ,  0.04779,  0.00758,  0.03593,
       -0.02769, -0.02174,  0.02441, -0.00663,  0.01617, -0.03903],
      dtype=float32)

In [17]:
user_embedding_layer(0)

<tf.Tensor: shape=(30,), dtype=float32, numpy=
array([-0.03207, -0.02283,  0.03549,  0.02813, -0.03413,  0.03188,
       -0.03514, -0.00757, -0.01221,  0.04228, -0.04017, -0.00138,
        0.04423,  0.0338 ,  0.04379, -0.04326,  0.04124, -0.0429 ,
        0.02943,  0.03373,  0.00597,  0.02351,  0.03395,  0.00442,
       -0.01538, -0.01041,  0.0054 ,  0.0008 ,  0.0071 , -0.04525],
      dtype=float32)>

In [18]:
user_embedding_layer(1)

<tf.Tensor: shape=(30,), dtype=float32, numpy=
array([-0.00124,  0.02806,  0.01952,  0.01807, -0.01903,  0.02258,
        0.03024, -0.04098, -0.02558, -0.00899, -0.01783, -0.04243,
       -0.01648,  0.02209, -0.00038,  0.00113, -0.04222,  0.00695,
        0.04144,  0.04491,  0.0142 ,  0.04779,  0.00758,  0.03593,
       -0.02769, -0.02174,  0.02441, -0.00663,  0.01617, -0.03903],
      dtype=float32)>

### (3) Item Embedding, User Embedding 구하기

Tensorflow Keras에서 Item Embedding의 값과 User Embedding의 값을 가져오는 것은 매우 간단합니다. 우리는 층의 연결을 통해 가져올 수 있습니다.

#### Item Embedding 구하기

이 때 주의해야 하는 것은 positive item과 negative item은 같은 임베딩 레이어에서 가져와야 합니다. <br>

In [19]:
pos_item_embedding = item_embedding_layer(pos_item_id)
neg_item_embedding = item_embedding_layer(neg_item_id)

In [20]:
pos_item_embedding

<tf.Tensor 'item_embedding/embedding_lookup/Identity_1:0' shape=(None, 31) dtype=float32>

In [21]:
neg_item_embedding

<tf.Tensor 'item_embedding/embedding_lookup_1/Identity_1:0' shape=(None, 31) dtype=float32>

#### User Embedding 구하기

유저 임베딩에서 우리는 마지막 임베딩에 1을 추가해주어야 합니다. 아이템 임베딩의 마지막 원소값 Bias를 추가하기 위함입니다.바로 아래와 같은 방식으로 유저 임베딩과 아이템 임베딩이 형성됩니다.

$$
U = [u_1, u_2, u_3, ..., u_{60}, 1] \\
I = [i_1, i_2, i_3, ..., i_{60}, i_{bias}] 
$$

Dot 연산으로 Bias 연산까지 같이 수행하기 위해 아래와 같이 코드를 작성하게 됩니다.

In [22]:
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import backend as K

user_embedding = user_embedding_layer(user_id)
one_embedding = K.ones_like(user_embedding[:,-1:])

user_embedding = Concatenate()([user_embedding, one_embedding])

### (4) Score 계산하기

우리는 고객이 본 아이템에 대한 Score와 고객이 보지 않은 아이템에 대한 Score의 차이가 극대화되도록 학습하게 됩니다. 이를 위해 BPR에서는 **Individual Probability**, 즉 고객이 본 아이템에 대해 보지 않은 아이템보다 선호할 확률을 구하게 됩니다.


In [23]:
from tensorflow.keras.layers import Dot
from tensorflow.keras.activations import sigmoid

pos_score = Dot(axes=(1,1))([user_embedding, pos_item_embedding])
neg_score = Dot(axes=(1,1))([user_embedding, neg_item_embedding])

diff_score = pos_score - neg_score

probs = sigmoid(diff_score)

In [24]:
probs

<tf.Tensor 'Sigmoid:0' shape=(None, 1) dtype=float32>

Bayesian Personalized Ranking은 위의 확률이 100%가 되도록 학습합니다.

### (5) Model 구성하기

입력값은 크게 세가지 `user_id`, `pos_item_id`,`neg_item_id`으로 나뉘어집니다. 그리고 출력값은 보지 않은 아이템에 대한 선호도보다 본 아이템에 대한 선호도가 높을 확률(`individual probability`)인 `probs`이 됩니다.

In [25]:
from tensorflow.keras.models import Model

model = Model([user_id, pos_item_id, neg_item_id], probs)

### (6) Regularization 적용하기

Matrix Factoriation은 쉽게 Overfitting, 즉 학습 데이터에만 과적합되는 현상이 발생합니다. 이를 방지하기 위해 가장 기본적인 방법론 중 하나는 Weight Decay, 즉 weight의 값이 너무 커지지 않도록 방지하는 것입니다. 이를 위해 아래와 같이 Loss를 추가해주게 되면, weight가 어느정도 줄어드는 방향으로 모델이 학습하게 됩니다.

In [26]:
l2_pos_item = pos_item_embedding**2
l2_neg_item = neg_item_embedding**2
l2_user = user_embedding**2
l2_reg = 0.0001

weight_decay = l2_reg * tf.reduce_sum(l2_pos_item + l2_neg_item + l2_user)

model.add_loss(weight_decay)

### (7) Model 컴파일하기

이전 구현체인 `implicit`에서는 기본 `SGD`를 이용했지만, 여기에서는 최대한 빠르게 수렴시키기 위해 변형체인 `Adagrad`를 이용하도록 하겠습니다.

In [27]:
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

model.compile(Adagrad(1.), 
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

## 2. 학습 데이터 구성하기

우리가 가지고 있는 데이터는 고객이 특정 영화에 대해 몇점의 평점을 주었는지에 대한 데이터입니다. 우리는 고객이 평가하지 않은 영화에 대한 정보를 생성해서 Pair 단위로 모델을 학습해야 합니다.

### (1) 고객이 평가하지 않은, 구매하지 않은 영화군 정의하기

In [28]:
# 유저별 평가한 영화목록 구성하기
itemset_per_user = (
    train_df
    .groupby('user_id')
    ['item_id']
    .apply(frozenset)
)

total_items = set(train_df.item_id.unique())

# 유저가 평가하지 않은 영화목록 구성하기
notseen_itemset_per_user = total_items - itemset_per_user
notseen_itemset_per_user = notseen_itemset_per_user.apply(list)

### (2) 학습 데이터 구성하기

우리는 매 Epoch마다 본것과 보지 않은 것에 대한 쌍을 무작위로 추출합니다. 그리고 학습 데이터에서 출력값은 항상 1로 나와야 합니다.(보지 않은 것에 대한 본것의 확률 = 100%)

In [29]:
# BPR
######################################################################
# def get_bpr_dataset(train_df, notseen_itemset_per_user):
#     batch_train_df = train_df.copy()
#     batch_train_df = batch_train_df.sample(frac=1)
#     batch_train_df['negative_item'] = batch_train_df.user_id.apply(
#         lambda x : random.choice(notseen_itemset_per_user[x]))
#    
#     X = {
#         "user":batch_train_df['user_id'].values,
#         "positive_item":batch_train_df['item_id'].values,
#         "negative_item":batch_train_df['negative_item'].values
#     }
#     y = np.ones((len(batch_train_df),1))
#    
#     return X, y

# WBPR
######################################################################
def get_bpr_dataset(train_df):
    batch_train_df = train_df.copy()
    batch_train_df = batch_train_df.sample(frac=1)

#     itemset_per_user = batch_train_df.groupby('user_id').item_id.apply(frozenset)
#     neg_samples = []
#     for user_id in tqdm(batch_train_df.user_id, total=len(batch_train_df)):
#         neg_sample = (
#             batch_train_df
#             .loc[~batch_train_df.item_id.isin(itemset_per_user[user_id]), 'item_id']
#             .sample(1)
#             .iloc[0]
#         )
#         neg_samples.append(neg_sample)
#     batch_train_df['negative_item'] = neg_samples

    batch_train_df['negative_item'] = batch_train_df.item_id.sample(frac=1).values
    
        
    X = {
        "user":batch_train_df['user_id'].values,
        "positive_item":batch_train_df['item_id'].values,
        "negative_item":batch_train_df['negative_item'].values
    }
    y = np.ones((len(batch_train_df),1))
    
    return X, y

In [30]:
#X,y = get_bpr_dataset(train_df, notseen_itemset_per_user)
X,y = get_bpr_dataset(train_df)

### (3) 모델 학습하기

epoch 10번에 걸쳐 모델을 학습시키도록 하겠습니다. 매 Epoch마다 새로운 학습 pair를 생성하도록 하였습니다. 

In [31]:
epoch = 10
for i in range(1,epoch+1):
    print("{}th".format(i))
    # X,y = get_bpr_dataset(train_df, notseen_itemset_per_user)
    X,y = get_bpr_dataset(train_df)
    model.fit(X, y, batch_size=64, verbose=2)

1th
1533/1533 - 1s - loss: 0.7085 - binary_accuracy: 0.4702
2th
1533/1533 - 1s - loss: 0.7070 - binary_accuracy: 0.4748
3th
1533/1533 - 1s - loss: 0.6958 - binary_accuracy: 0.5363
4th
1533/1533 - 1s - loss: 0.6555 - binary_accuracy: 0.6331
5th
1533/1533 - 1s - loss: 0.6162 - binary_accuracy: 0.6755
6th
1533/1533 - 1s - loss: 0.5930 - binary_accuracy: 0.6975
7th
1533/1533 - 1s - loss: 0.5776 - binary_accuracy: 0.7115
8th
1533/1533 - 1s - loss: 0.5666 - binary_accuracy: 0.7224
9th
1533/1533 - 1s - loss: 0.5568 - binary_accuracy: 0.7323
10th
1533/1533 - 1s - loss: 0.5499 - binary_accuracy: 0.7392


## 3. 모델 평가하기

Hit Ratio를 계산하기 위해서 User, Item의 쌍 별로 Score가 얼마나 나왔는지를 계산해야 합니다. Score을 계산하는 것은 BPR 모델에서 Positive Score와 Negative Score를 계산하는 것으로 이루어집니다. 이부분을 떼어내어 새로 모델을 구성하도록 하겠습니다.



In [32]:
test_model = Model([user_id,pos_item_id],pos_score)

Hit Ratio를 계산하기 위해서 우리는 본 영화에 대한 Score와 보지 않은 영화에 대한 Score를 계산해야 합니다. 이때 본 영화에 대한 Score가 보지 않은 영화에 대한 Score보다 높을수록 좋습니다. 보통 Top-10, 보지 않은 영화 100개 중 10번째 Score보다 높으면 올바르게 추론(Hit)했다고 계산합니다. 이에 따라 계산하면 아래와 같이 계산됩니다.

In [33]:
hit = 0.
for i, row in tqdm(hit_ratio_df.iterrows()):
    user = np.array([row.user_id])
    seens = np.array([row.item_id])
    pos_scores = test_model.predict([user,seens])
    pos_scores = pos_scores[0,0]
    
    not_seens = np.array(row.not_seen_list)
    users = np.array([row.user_id]*len(not_seens))   
    neg_scores = test_model.predict([users,not_seens])
    
    if pos_scores > np.sort(neg_scores.flatten())[-10]:
        hit += 1

943it [00:48, 19.26it/s]


In [34]:
hit_ratio = hit / len(hit_ratio_df)        
print(f"hit ratio : {hit_ratio:.3f}")

hit ratio : 0.513


## 4. 모델 이용하기

Bayesian Personalized Ranking에서 학습된 임베딩 값들은 아래와 같은 방식으로 임베딩 값을 추출할 수 있습니다. 간단하게 어떻게 이용했는지를 복습해도록 하겠습니다.

In [35]:
item_embedding_weight = (
    model.get_layer('item_embedding').get_weights()[0])

item_embedding_df = pd.DataFrame(item_embedding_weight)
item_embedding_df = item_embedding_df[
    item_embedding_df.index.isin(movies_df.item_id)]
item_embedding_df.index = movies_df.title

In [36]:
(
    item_embedding_df
    .dot(item_embedding_df.loc['Copycat (1995)'])
    .sort_values(ascending=False)
    .iloc[:10]
)

title
Tales From the Crypt Presents: Demon Knight (1995)    1.803866
Natural Born Killers (1994)                           1.803743
Candyman (1992)                                       1.685850
Copycat (1995)                                        1.671408
Jaws 2 (1978)                                         1.564870
Virtuosity (1995)                                     1.512189
Demolition Man (1993)                                 1.510551
Bad Boys (1995)                                       1.509586
Nightmare on Elm Street, A (1984)                     1.476377
Crow, The (1994)                                      1.475660
dtype: float32

In [37]:
(
    item_embedding_df
    .dot(item_embedding_df.loc['Die Hard 2 (1990)'])
    .sort_values(ascending=False)
    .iloc[:10]
)

title
Demolition Man (1993)                   2.346600
Cliffhanger (1993)                      2.256768
Under Siege (1992)                      2.227391
Die Hard: With a Vengeance (1995)       2.210665
Young Guns II (1990)                    2.172642
Three Musketeers, The (1993)            2.059460
Die Hard 2 (1990)                       2.057950
Under Siege 2: Dark Territory (1995)    2.054211
Stargate (1994)                         2.044352
Crow, The (1994)                        2.043220
dtype: float32

In [38]:
(
    item_embedding_df
    .dot(item_embedding_df.loc['Terminator, The (1984)'])
    .sort_values(ascending=False)
    .iloc[:10]
)

title
Good, The Bad and The Ugly, The (1966)                                         1.224539
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)    1.108616
Blade Runner (1982)                                                            0.981658
Terminator, The (1984)                                                         0.981235
Die Hard (1988)                                                                0.955278
2001: A Space Odyssey (1968)                                                   0.934000
Braveheart (1995)                                                              0.899696
Glory (1989)                                                                   0.884793
Gandhi (1982)                                                                  0.876719
Monty Python and the Holy Grail (1974)                                         0.872174
dtype: float32

<hr>
<div style = "background-image: url('https://pai-picture.s3.ap-northeast-2.amazonaws.com/PAI-Identity/PublicAI+Logo.png');background-repeat: no-repeat; background-position: right; background-size: 60px 40px; padding : 5px 70px 5px 5px;">
    Copyright(c) 2020 by Public AI. All rights reserved.<br>
    Writen by PAI, SeonYoul Choi ( best10@publicai.co.kr )  last updated on 2020/06/22
</div>
<hr>